# CNN with Cifar10

Example of Convolutional Neural Net (CNN) with TensorFlow (TF) 2.0.

This example builds a CNN from scratch, with custom layers and training.

**Sources:**

* [Udemey_tf_course](https://www.udemy.com/course/complete-guide-to-tensorflow-for-deep-learning-with-python/)
* [TF_tutorial](https://www.tensorflow.org/tutorials/customization/custom_training)
* [Medium_blog](https://becominghuman.ai/image-classification-with-tensorflow-2-0-without-keras-e6534adddab2)

`Author: Rodrigo Vimieiro`

`Date: Apr, 2020`

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/LAVI-USP/Machine-Learning/blob/master/Deep%20Learning/Classifiers/CNN_cifar10_TF2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/LAVI-USP/Machine-Learning/blob/master/Deep%20Learning/Classifiers/CNN_cifar10_TF2.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets
import matplotlib.pyplot as plt
import numpy as np

## Import dataset

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

## Helper functions

In [ ]:
# Source: https://www.udemy.com/course/complete-guide-to-tensorflow-for-deep-learning-with-python/
class CifarHelper():
    
    def __init__(self):
        
        self.i = 0
        
        # Intialize some empty variables for later on
        self.training_images = None
        self.training_labels = None
        
        self.test_images = None
        self.test_labels = None
    
    def set_up_images(self):
        
        print("Setting Up Training Images and Labels")
        
        self.training_images = train_images / 255.0
        self.training_labels = self.one_hot_encode(train_labels) 
        
        print("Setting Up Test Images and Labels")
        
        self.test_images = test_images / 255.0
        self.test_labels = self.one_hot_encode(test_labels)

    def one_hot_encode(self, vec):

        n = len(vec)
        out = np.zeros((n, 10))
        for i in range(n):
            out[i, vec[i]] = 1
        
        return out

        
    def next_batch(self, batch_size):
        
        x = self.training_images[self.i:self.i+batch_size]
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_images)
        return x, y

In [ ]:
ch = CifarHelper()
ch.set_up_images()

## Layer functions

In [ ]:
# Convolutional layer
def conv_layer(input_x,w,b):
  
  # input_x -> [batch,H,W,Channels]
  # filter_shape -> [filters H, filters W, Channels In, Channels Out]

  y = tf.nn.conv2d(input=input_x,filters=w,strides=[1,1,1,1],padding='SAME') + b

  y = tf.nn.relu(y)

  return y

In [ ]:
# Pooling layer
def maxPool_layer(x,poolSize):
  # x -> [batch,H,W,Channels]

  return tf.nn.max_pool2d(input=x,ksize=[1,poolSize,poolSize,1],strides=[1,poolSize,poolSize,1],padding="SAME")

In [ ]:
# Fully connected layer
def fullyConnected_layer(input_layer,w,b):

  y = tf.matmul(input_layer,w) + b

  return y

## Creating the Model

In [ ]:
def get_tfVariable(shape, name):

  return tf.Variable(tf.random.truncated_normal(shape,stddev=0.1), name=name, trainable=True, dtype=tf.float32)

class my_model():

  def __init__(self):
    
    self.pool_size = 2
    self.dropout = 0.5
    self.nclasses = 10

    self.shapes = [
    [5, 5, 3, 32], 
    [5, 5, 32, 64],
    [8*8*64,512],
    [512, self.nclasses]
    ]

    self.weights = []
    for i in range(len(self.shapes)):
      self.weights.append( get_tfVariable(self.shapes[i] , 'weight{}'.format( i ) ) )

    self.bias = []
    for i in range(len(self.shapes)):
      self.bias.append( get_tfVariable([1,self.shapes[i][-1]] , 'bias{}'.format( i ) ) )



  def run(self, x_input):
    
    conv1 = conv_layer(x_input,self.weights[0],self.bias[0]) 
    pool1 = maxPool_layer(conv1,poolSize=self.pool_size)
    
    conv2 = conv_layer(pool1,self.weights[1],self.bias[1]) 
    pool2 = maxPool_layer(conv2,poolSize=self.pool_size)
    
    flat1 = tf.reshape(pool2,[-1,pool2.shape[1]*pool2.shape[2]*pool2.shape[3]])
    
    fully1 = tf.nn.relu(fullyConnected_layer(flat1,self.weights[2],self.bias[2]))
    
    fully1_dropout = tf.nn.dropout(fully1,rate=self.dropout)
    
    y_pred = fullyConnected_layer(fully1_dropout,self.weights[3],self.bias[3])
    
    #print(conv1.shape,pool1.shape,conv2.shape,pool2.shape,flat1.shape,fully1.shape,y_pred.shape)

    return y_pred

  def trainable_variables(self):

    return self.weights + self.bias

In [ ]:
model = my_model()

## Creating loss function





In [ ]:
def loss_function(y_pred,y_true):
    
    return tf.nn.softmax_cross_entropy_with_logits(labels=tf.stop_gradient(y_true),logits=y_pred)

## Creating optimizer

In [ ]:
optimizer = tf.optimizers.Adam(learning_rate=0.001)

## Trainning function

In [ ]:
def train_step( model, x_input , y_true, epoch):

  epoch_accuracy = None
  epoch_loss_avg = None
    
  with tf.GradientTape() as tape:
        
    # Get the predictions
    preds = model.run(x_input)
        
    # Calc the loss
    current_loss = loss_function(preds,y_true)
    
    # Get the gradients
    grads = tape.gradient(current_loss, model.trainable_variables())
    
    # Update the weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables()))
    
    if epoch%100 == 0:

      y_pred = model.run(ch.test_images)
      matches  = tf.equal(tf.math.argmax(y_pred,1), tf.math.argmax(ch.test_labels,1))

      epoch_accuracy = tf.reduce_mean(tf.cast(matches,tf.float32))
      epoch_loss_avg = tf.reduce_mean(current_loss)

      print("--- On epoch {} ---".format(epoch))
      tf.print("Accuracy: ", epoch_accuracy, "| Loss: ",epoch_loss_avg)
      print("\n")

    return epoch_accuracy,epoch_loss_avg
      

## Train model

In [ ]:
num_epochs = 5000
batch_size = 100

train_loss_results = []
train_accuracy_results = []

for epoch in range(num_epochs):
    
  # Get next batch
  batch_x, batch_y = ch.next_batch(batch_size)
    
  # Train the model
  epoch_accuracy, epoch_loss_avg = train_step(model, batch_x, batch_y, epoch)

  if(epoch_loss_avg is not None):
    train_loss_results.append(epoch_loss_avg)
    train_accuracy_results.append(epoch_accuracy)

plt.plot(train_loss_results)
plt.title('Loss')
plt.show()
plt.title('Accuracy')
plt.plot(train_accuracy_results)
plt.show()

In [ ]:
n = 784
pred = model.run(ch.test_images[n:n+1])
tf.print(tf.math.argmax(pred,1))
tf.print(tf.math.argmax(ch.test_labels[n:n+1],1))